# OUTPUT CONVERTER

This notebook can be used to convert the probability densities files provided by QSLE-v1.0 program into a gnuplot readable version, that is easier to use for further analysis. In addition, the user can modify the units of measure, for example changing the radians into degrees.

Please read this notebook carefully and follow the procedure step by step.

## Import Libraries

To use this notebook, the following libraries need to be imported:
* `Numpy`
* `Scipy`
* `Pandas`
* `Tkinter`

After having installed the mentioned libraries, you can run the following cells.

In [ ]:
import numpy as np
import scipy.constants as sc
import pandas as pd
import tkinter as tk
from tkinter import filedialog

def error_exit():
    print("\n### Terminating program because of errors ###\n")
    exit(1)

print("\n********************************")
print("* LIBRARIES LOADED SUCCESSFULLY*")
print("********************************")

## Choosing The Restart File

The first thing you must do is to select the restart file printed from the QSLE run (i.e. `PREFIX_restart.inp`).

Run the following cell and choose the right *.inp filename.

In [ ]:
root=tk.Tk()
root.withdraw()
restart= filedialog.askopenfilename(title="CHOOSE THE RESTART FILE", filetypes=(("restart files","*.inp"),("all files", "*.*")))

input_file=pd.read_csv(restart, sep='\s+', header=None, usecols=[0,1])

angle_points= int(input_file[input_file[0].str.contains("Angle_points")].iat[0,1])
mom_points= int(input_file[input_file[0].str.contains("Momentum_points")].iat[0,1])
mom_max= float(input_file[input_file[0].str.contains("Mom_max")].iat[0,1])
prefix= input_file[input_file[0].str.contains("Prefix")].iat[0,1]

mom_grid=np.linspace(-mom_max,mom_max,mom_points)
angle_grid=np.zeros(angle_points)
for i in range(angle_points):
    angle_grid[i]=-np.pi+i*2.*np.pi/angle_points

print("\n*************************************")
print("* RESTART FILE IMPORTED SUCCESSFULLY*")
print("*************************************")

## Choosing The Units Of Measure

Now, you can set the units of measure you want to adopt. 

Run the following cell and follow the instructions on the appearing windows.

In [ ]:
list_titles=["ANGLE", "LENGTH", "TIME", "MASS"]
list_selections=["RADIAN", "ANGSTROM", "FEMTOSECOND", "DALTON"]
list_choices=[["RADIAN", "DEGREE"],
              ["ANGSTROM", "MICROMETER", "NANOMETER", "PICOMETER", "FEMTOMETER", "AU_LENGTH", "SI_LENGTH", "CGS_LENGTH"],
              ["FEMTOSECOND", "NANOSECOND", "PICOSECOND","ATTOSECOND", "AU_TIME", "SI_TIME", "CGS_TIME"],
              ["DALTON", "AU_MASS", "SI_MASS", "CGS_MASS"]]

list_coeff=[[1.0, 180.0/sc.pi],
              [1.0, 10**(-4), 10**(-1), 10**(2), 10**(5), 10**(-10)/(sc.Planck**2/(4 * sc.pi**2 * sc.m_e * sc.e**2)), 10**(-10), 10**(-8)],
              [1.0, 10**(-6), 10**(-3), 10**(3), 10**(-15)*(4 * sc.pi * sc.c * sc.Rydberg), 10**(-15), 10**(-15)],
              [1.0,  sc.atomic_mass /sc.m_e, sc.atomic_mass, sc.atomic_mass*1000]]


print("\n********************************")
print("* STARTING THE CHOICE OF UNITS *")
print("********************************")


for i in range(len(list_selections)):
    appear = True
    def showSelected():
        global appear
        global list_selections
        global i
        show.config(text=str(bigListbox.get("anchor"))+ " is your current choice.\nIf you are sure, press NEXT.")
        list_selections[i]=str(bigListbox.get("anchor"))
        if appear == True:
            show.pack()
            b1.pack(pady=10)
            appear = False
    def procede():
        rootWindow.quit()
        rootWindow.destroy()

    bigArr=list_choices[i]

    rootWindow = tk.Tk()
    rootWindow.title(list_titles[i] + " UNIT")
    height = 150+25*len(bigArr)
    rootWindow.geometry("300x" +str(height))

    bigListbox = tk.Listbox(rootWindow, height=len(bigArr), selectmode= "single", font=10)
    bigListbox.pack(pady=10)

    myIndex = len(bigArr)
    for index in range(0, myIndex):
        bigListbox.insert(myIndex, bigArr[index])

    tk.Button(rootWindow, text='SELECT HIGHLIGHTED KEYWORD', command=showSelected).pack(pady=10)
    show = tk.Label(rootWindow)
    show.config(text= list_selections[i] +" is your current choice.\nIf you are sure, press NEXT.")

    b1 = tk.Button(rootWindow, text='NEXT', command=procede)

    rootWindow.mainloop()

    print("+ {0} --> {1}".format(list_titles[i],list_selections[i]))

angle_converter= 1.0
if list_selections[0]=="DEGREE":
    angle_converter = 180.0/sc.pi

mom_converter = 1.0
for i in range(len(list_selections)):
    choices = list_choices[i]
    coeff = list_coeff[i]
    for ii in range(len(choices)):
        if list_selections[i] == choices[ii]:
            if list_titles[i] == "ANGLE":
                mom_converter *= coeff[ii]
            if list_titles[i] == "LENGTH":
                mom_converter *= (coeff[ii]**2)
            if list_titles[i] == "TIME":
                mom_converter /= coeff[ii]
            if list_titles[i] == "MASS":
                mom_converter *= coeff[ii]
            break


print("\n*******************************")
print("* UNITS CONVERTED SUCCESSFULLY*")
print("*******************************")

## Choosing The 'Time' Files

Finally, you can select the output files related to the evolution of the probability density (i.e. `PREFIX_time_N.dat`).

Run the following cell and choose the *.dat files of your choice (for multiple selection remember to hold down CTRL).

In [ ]:
files= filedialog.askopenfilenames(title="CHOOSE THE 'TIME' FILES", filetypes=(("data files","*.dat"),("all files", "*.*")))
directory = files[0][:files[0].rfind('/')]
initial_path= directory + "/Extended_"

grid=np.zeros((angle_points*mom_points,2))
for i in range(angle_points):
    for ii in range(mom_points):
        grid[i*mom_points + ii,0]=angle_grid[i] * angle_converter
        grid[i*mom_points + ii,1]=mom_grid[ii] * mom_converter

print("\n************************************")
print("* STARTING THE CONVERSION OF FILES *")
print("************************************")

for file in files:
    data=np.genfromtxt(file, dtype=np.float32)
    data /= (angle_converter*mom_converter)
    name= file[file.rfind(prefix):]
    with open(initial_path + name, "w") as output:
        for i in range(len(grid)):
            output.write("{0} {1} {2} {3}\n".format(grid[i,0], grid[i,1], data[i], data[i+angle_points*mom_points]))
            if i%mom_points == mom_points-1:
                output.write("\n")
        print("+ {0} --> Extended_{0}".format(name))


print("\n*******************************")
print("* FILES CONVERTED SUCCESSFULLY*")
print("*******************************")


## Structure of the new output

In this way, new files called `Extended_PREFIX_time_N.dat` are generated, which can be easily displayed using `gnuplot platform`:

`sp "Extended_PREFIX_time_N.dat" u 1:2:3 w pm3` -> to generate the 3D plot of the ground state

`sp "Extended_PREFIX_time_N.dat" u 1:2:4 w pm3` -> to generate the 3D plot of the excited state

The structure of the new file is here described:

* 1st column) ANGLE

* 2nd column) MOMENTUM

* 3rd column) GROUND STATE PROBABILITY DENSITY

* 4th column) EXCITED STATE PROBABILITY DENSITY